# Testing Notebook


Random online python phenology functions that might help:

- https://gist.github.com/YanCheng-go/d4e17831f294199443d0f7682558e608

- https://github.com/JavierLopatin/PhenoPY



In [ ]:
# !pip install richdem
# !pip install xarray --upgrade

In [1]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
from odc.algo import xr_reproject
import hdstats
import numpy as np
import pandas as pd
import sys
import xarray as xr
import datetime as dt
import os

sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_bandindices import calculate_indices
from deafrica_plotting import display_map, rgb
from deafrica_temporal_statistics import xr_phenology, temporal_statistics, fast_completion, smooth
from datacube.utils.geometry import assign_crs
from deafrica_dask import create_local_dask_cluster

import warnings
warnings.filterwarnings("ignore", "Mean of empty slice")
warnings.simplefilter("ignore", FutureWarning)

%load_ext autoreload
%autoreload 2

In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:40795 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


### Connect to the datacube

In [3]:
dc = datacube.Datacube(app='Vegetation_phenology')

### Analysis parameters


In [6]:
# Set the vegetation proxy to use
veg_proxy = 'NDVI'

# Define area of interest
lat = -34.288#22.817 #-34.288 
lon = 20.012#28.518 #20.012 
lon_buffer = 0.35#0.0175
lat_buffer = 0.35#0.004

# Combine central lat,lon with buffer to get area of interest
lat_range = (lat-lat_buffer, lat+lat_buffer)
lon_range = (lon-lon_buffer, lon+lon_buffer)

# Set the range of dates for the analysis
years_range = ('2018-01', '2018-06')

## View the selected location

In [7]:
# display_map(x=lon_range, y=lat_range)

## Load cloud-masked Sentinel-2 data

The first step is to load Sentinel-2 data for the specified area of interest and time range. 
The `load_ard` function is used here to load data that has been masked for cloud, shadow and quality filters, making it ready for analysis.

In [8]:
# Create a reusable query
query = {
    'y': lat_range,
    'x': lon_range,
    'time': years_range,
    'measurements': ['blue', 'green', 'red', 'nir_1'],
    'resolution': (-20,20),
    'output_crs': 'epsg:6933'
}

# Load available data from Landsat 8
ds = load_ard(dc=dc,
              products=['s2_l2a'],
              dask_chunks={'x':1000, 'y':1000,'time':-1},
              **query
              )

# # Load available data from Landsat 8
# ds1 = load_ard(dc=dc,
#               products=['s2_l2a'],
#               **query
#               )

print(ds)

Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Applying pixel quality/cloud mask
Returning 71 time steps as a dask array
<xarray.Dataset>
Dimensions:      (time: 71, x: 3378, y: 3706)
Coordinates:
    spatial_ref  int32 6933
  * x            (x) float64 1.897e+06 1.897e+06 ... 1.965e+06 1.965e+06
  * y            (y) float64 -4.086e+06 -4.087e+06 ... -4.161e+06 -4.161e+06
  * time         (time) datetime64[ns] 2018-01-01T08:42:22 ... 2018-06-30T08:...
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(71, 1000, 1000), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(71, 1000, 1000), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(71, 1000, 1000), meta=np.ndarray>
    nir_1        (time, y, x) float32 dask.array<chunksize=(71, 1000, 1000), meta=np.ndarray>
Attributes:
    crs:           epsg:6933
    grid_mapping:  spatial_ref


**Once the load is complete**, we can plot the data as a true-colour image using the `rgb` function.  

In [ ]:
# rgb(ds, index=[0,5], col_wrap=1)

In [11]:
# Calculate the chosen vegetation proxy index and add it to the loaded data set
# ds = (ds.nir - ds.red)/(ds.nir + ds.red)
ds = calculate_indices(ds, index=veg_proxy, collection='s2')
# ds1 = calculate_indices(ds1, index=veg_proxy, collection='s2')
# ds

In [9]:
stats=['discordance','abs_change','complexity','f_mean','central_diff']

In [ ]:
x = temporal_statistics(ds1.NDVI, stats=stats)
x

In [ ]:
%time
y = temporal_statistics(ds.NDVI, stats=stats).compute()
y

In [ ]:
z = x - y

In [ ]:
x.discordance.plot()

In [ ]:
y.discordance.plot()

In [ ]:
z.discordance.plot()

In [12]:
%%time
phen = xr_phenology(ds.NDVI,
                    method_sos='median',
                    method_eos='median',
                    complete='linear',
                    smoothing='rolling_mean').compute()
phen

CPU times: user 56.1 s, sys: 4.73 s, total: 1min
Wall time: 17min 34s


<xarray.Dataset>
Dimensions:      (x: 3378, y: 3706)
Coordinates:
  * x            (x) float64 1.897e+06 1.897e+06 ... 1.965e+06 1.965e+06
  * y            (y) float64 -4.086e+06 -4.087e+06 ... -4.161e+06 -4.161e+06
    spatial_ref  int32 0
Data variables:
    SOS          (y, x) int16 46 73 73 61 73 11 3 71 ... 26 48 26 38 96 6 51 18
    POS          (y, x) int16 111 111 106 111 106 96 171 ... 68 68 173 98 98 166
    EOS          (y, x) int16 146 161 168 141 156 161 ... 101 106 1 131 138 171
    Trough       (y, x) float32 0.0893186 -0.010265502 ... -0.96721315
    vSOS         (y, x) float32 0.3694118 0.35187852 ... -0.062031705
    vPOS         (y, x) float32 0.49560723 0.46754968 ... 0.33333334 0.28395063
    vEOS         (y, x) float32 0.41601783 0.37746027 ... -0.07184811 0.0
    LOS          (y, x) int16 100 88 95 80 83 150 179 ... 75 68 86 125 87 153
    AOS          (y, x) float32 0.40628862 0.47781518 ... 1.2955976 1.2511637
    ROG          (y, x) float32 0.0019414682 0.0030439778 ... 0.0023377186
    ROS          (y, x) float32 -0.0022739829 -0.0018017882 ... -0.056790125
Attributes:
    grid_mapping:  spatial_ref

In [ ]:
%%time
phen1 = xr_phenology(ds1.NDVI,
                    method_sos='median',
                    method_eos='median',
                    complete='fast_complete',
                    smoothing='wiener')
phen1

In [ ]:
z  = phen - phen1

In [ ]:
phen.SOS.plot()

In [ ]:
phen1.SOS.plot()

In [ ]:
z.SOS.plot()

In [ ]:
i_complete=fast_completion(i.NDVI)

In [ ]:
i_complete.mean(['x', 'y']).plot()

In [ ]:
x=smooth(i_complete)

In [ ]:
x.mean(['x', 'y']).plot()

In [ ]:
z = I_mapblocks - i_complete

In [ ]:
template=i.NDVI.drop('spatial_ref')

I_mapblocks = i.NDVI.map_blocks(
    fast_completion,
    template=template)

# I_mapblocks

In [ ]:
template=I_mapblocks

I_mapblocks_smooth = I_mapblocks.map_blocks(
    smooth,
    template=template).compute()

In [ ]:
I_mapblocks_smooth.mean(['x', 'y']).plot()

In [ ]:
j = I_mapblocks_smooth - x

In [ ]:
j.mean(['x', 'y']).plot()

In [ ]:
# def poly_fit(time, data, degree):
    
#     pfit = np.polyfit(time, data, degree) 
    
#     return np.transpose(np.polyval(pfit,time))

# def poly_fit_smooth(time, data, degree, n_pts):
#         """
#         """
    
#         time_smooth_inds = np.linspace(0, len(time), n_pts)
#         time_smooth = np.interp(time_smooth_inds, np.arange(len(time)), time)

#         data_smooth = np.array([np.array([coef * (x_val ** current_degree) for
#                                 coef, current_degree in zip(np.polyfit(time, data, degree),
#                                 range(degree, -1, -1))]).sum() for x_val in time_smooth])

#         return data_smooth

# def xr_polyfit(doy,
#                da,
#                degree,
#                interp_multiplier=1):    
    
#     # Fit polynomial curve to observed data points
#     if interp_multiplier==1:
#         print('Fitting polynomial curve to existing observations')
#         pfit = xr.apply_ufunc(
#             poly_fit,
#             doy,
#             da, 
#             kwargs={'degree':degree},
#             input_core_dims=[["time"], ["time"]], 
#             output_core_dims=[['time']],
#             vectorize=True,  
#             dask="parallelized",
#             output_dtypes=[da.dtype],
#         )
    
#     if interp_multiplier > 1:
#         print("Fitting polynomial curve to "+str(len(doy)*interp_multiplier)+
#                                                       " interpolated points")
#         pfit = xr.apply_ufunc(
#             poly_fit_smooth,  # The function
#             doy,# time
#             da,#.chunk({'time': -1}), #the data
#             kwargs={'degree':degree, 'n_pts':len(doy)*interp_multiplier},
#             input_core_dims=[["time"], ["time"]], 
#             output_core_dims=[['new_time']], 
#             output_sizes = ({'new_time':len(doy)*interp_multiplier}),
#             exclude_dims=set(("time",)),
#             vectorize=True, 
#             dask="parallelized",
#             output_dtypes=[da.dtype],
#         ).rename({'new_time':'time'})
    
#         # Map 'dayofyear' onto interpolated time dim
#         time_smooth_inds = np.linspace(0, len(doy), len(doy)*interp_multiplier)
#         new_datetimes = np.interp(time_smooth_inds, np.arange(len(doy)), doy)
#         pfit = pfit.assign_coords({'time':new_datetimes})
    
#     return pfit

# # da=xr_polyfit(dayofyear=dayofyear, 
# #               da=da,
# #               degree=degree,
# #               interp_multiplier=interp_multiplier)

In [ ]:
# #set up figure
# fig, ax = plt.subplots(nrows=5,ncols=2,figsize=(18,25), sharex=True, sharey=True)

# #start of season
# temp_stats.discordance.plot(ax=ax[0,0])
# ax[0,0].set_title('discordance')
# temp_stats.f_std.plot(ax=ax[0,1])
# ax[0,1].set_title('f_std')

# #peak of season
# temp_stats.f_mean.plot(ax=ax[1,0])
# ax[1,0].set_title('f_mean')
# phen.f_median.plot(ax=ax[1,1])
# ax[1,1].set_title('f_median')

# #end of season
# temp_stats.mean_change.plot(ax=ax[2,0])
# ax[2,0].set_title('mean_change')
# phen.med_change.plot(ax=ax[2,1])
# ax[2,1].set_title('med_change')

# #Length of Season
# temp_stats.abs_change.plot(ax=ax[3,0])
# ax[3,0].set_title('abs_change');

# #Amplitude
# temp_stats.complexity.plot(ax=ax[3,1])
# ax[3,1].set_title('complexity')

# #rate of growth
# temp_stats.central_diff.plot(ax=ax[4,0])
# ax[4,0].set_title('central_diff')

# #rate of Sensescence
# temp_stats.num_peaks.plot(ax=ax[4,1])
# ax[4,1].set_title('num_peaks');

# plt.tight_layout();